## This script is used for generating reduction.onnx model. For this first I have written custom_op_export() method. The first part is simply
evaluating the images for cosine similarities and producing the results. 

In [16]:
from numpy import dtype
import timm
import cv2
import os
import torch
from torchvision.models.feature_extraction import create_feature_extractor
import random
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
torch.ops.load_library("op.pyd")

from prepare_input import prepare

from render import show_images

model = timm.create_model('resnet18', pretrained=True)
model.eval();

current_path = os.getcwd()

folder = os.path.join(current_path, 'Challenge_images')
imgs = []

    # loop us implemented to read the images one by one from the loop.
    # listdir return a list containing the names of the entries
    # in the directory.
for filename in os.listdir(folder):
    img = cv2.imread((os.path.join(folder, filename)))
    img = cv2.resize(img, (416, 416),
               interpolation = cv2.INTER_NEAREST)
    imgs.append(img)

inps = [prepare(img, model.default_cfg['mean'], model.default_cfg['std'])
        for img in imgs]

return_nodes = ['layer1', 'layer2', 'layer3', 'layer4']

feat_ext = create_feature_extractor(model, return_nodes=return_nodes)
tensor_result =torch.zeros(50,512)
index = 0
for i in inps:
    with torch.no_grad():
        out = feat_ext(i)
    
    m = nn.AdaptiveAvgPool2d((1,1))
    n = nn.AdaptiveAvgPool2d((1,1))
    o = nn.AdaptiveAvgPool2d((1,1))
    p = nn.AdaptiveAvgPool2d((1,1))

    layer1_out = m(out['layer1']).view(1,-1)
    layer2_out = n(out['layer2']).view(1,-1)
    layer3_out = m(out['layer3']).view(1,-1)
    layer4_out = m(out['layer4']).view(1,-1)

    x = torch.ops.custom_namespace.op(layer1_out,layer2_out,layer3_out,layer4_out)
    tensor_result[index] = x
    index = index + 1
image_list =[]
for i in range(0,50):
    count = 0
    for j in range(0,50):
        cos = torch.nn.CosineSimilarity(dim=0 , eps = 1e-6)
        output = cos(tensor_result[i],tensor_result[j])
        if(output>=0.75):
            count = count + 1
        else:
            pass
    if count <= 1:
        image_list.append(i)
    
print("Unique images are:")
for x in image_list:
    print(x)







Unique images are:
1
11
39


## Import torch and torch.onnx for onnx runtime. 

In [72]:
import torch

import torch.onnx
import torch._C._onnx as _C_onnx

## Register the custom operator with ONNX runtime

In [4]:
def register_custom_op():
    def my_op(g, layerOne, layerTwo, layerThree, layerFour):
        return g.op("mydomain::testop", layerOne, layerTwo, layerThree, layerFour)

    from torch.onnx import register_custom_op_symbolic

    register_custom_op_symbolic("custom_namespace::op", my_op, 9)

## export_custom_op() method for exporting the model in .onnx format. 

In [114]:
from torch import ScriptModule


def export_custom_op():
      layerOne = torch.randn(1,64)
      layerTwo = torch.randn(1,128)
      layerThree = torch.randn(1,256)
      layerFour = torch.randn(1,512)
      class CustomModel(torch.nn.Module):
            def __init__(self):
                  super(CustomModel, self).__init__()
                  model = timm.create_model('resnet18', pretrained=True)
                  model.eval();
                  current_path = os.getcwd()

                  folder = os.path.join(current_path, 'Challenge_images')
                  imgs = []

    # loop us implemented to read the images one by one from the loop.
    # listdir return a list containing the names of the entries
    # in the directory.
                  for filename in os.listdir(folder):
                        img = cv2.imread((os.path.join(folder, filename)))
                        img = cv2.resize(img, (416, 416),
                                          interpolation = cv2.INTER_NEAREST)
                        imgs.append(img)

                  inps = [prepare(img, model.default_cfg['mean'], model.default_cfg['std'])
                              for img in imgs]

                  return_nodes = ['layer1', 'layer2', 'layer3', 'layer4']

                  feat_ext = create_feature_extractor(model, return_nodes=return_nodes)

                  tensor_result =torch.zeros(50,512)
                  index = 0
                  for i in inps:
                        with torch.no_grad():
                              out = feat_ext(i)
    
                  m = nn.AdaptiveAvgPool2d((1,1))
                  n = nn.AdaptiveAvgPool2d((1,1))
                  o = nn.AdaptiveAvgPool2d((1,1))
                  p = nn.AdaptiveAvgPool2d((1,1))

                  layer1_out = m(out['layer1']).view(1,-1)
                  layer2_out = n(out['layer2']).view(1,-1)
                  layer3_out = m(out['layer3']).view(1,-1)
                  layer4_out = m(out['layer4']).view(1,-1)
                  torch.ops.custom_namespace.op(layerOne, layerTwo, layerThree, layerFour)
            def forward(self ,x):
                 return None 
                  
      
      custom_model = CustomModel()

      
      torch.onnx.export(custom_model,args=None,f="reduction.onnx",
                      opset_version=15)
                      

In [5]:
torch.ops.load_library("op.pyd")

In [7]:
register_custom_op()

In [115]:
export_custom_op()